In [5]:
import requests
import csv 
from collections import Counter
from pandas import *

listoffiles=[]
listoffilmnames=[]
        
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False 
    
def csvgetter(user,password,*otherusernames):
    s=requests.session()  #this is the data that is sent to login to icheckmovies
    login_data={'login[username]': user,'login[password]': password,'submit': 'login',} #send the login data
    s.post("https://www.icheckmovies.com/login/", data=login_data) #csv link 'r'
    usernames_list=list(otherusernames)
    usernames_list.append(user)
    for item in usernames_list:
        url='https://www.icheckmovies.com/movies/checked/?user='+item+'&export'
        r=s.get(url)
        checklist=r.text
        checklist_encode = checklist.encode('utf-8')
        filename=item+'_checklist'+'.csv'
        file=open(filename, "w")
        file.write(checklist_encode)
        file.close() 
        listoffiles.append(filename)
    numberofusers=str((len(listoffiles)))
    print('There are '+numberofusers+' users in the database: '+'\n'+'\n'.join(listoffiles))
    
def listmaker(username):
    filmlist=[]
    toplist=[]
    mostchecked=[]
    csvname=username+"_checklist.csv"
    csv_reader = csv.reader(open(csvname, "rb"), delimiter=';')
    for row in csv_reader:
        filmlist.append(row[0])
        mostcheckednum=row[3]
        if is_number(mostcheckednum) is True:
            mostcheckednum=int(row[3])
        mostchecked.append(mostcheckednum)
        toplistnum=row[5]
        if is_number(toplistnum) is True:
            toplistnum=int(row[5])
        toplist.append(toplistnum)        
    filmlist.pop(0)
    mostchecked.pop(0)
    toplist.pop(0)
    compiled_dataframe={'title' : filmlist,
            'most_checked': mostchecked,
            'top_lists': toplist}
    userlist=DataFrame(compiled_dataframe)
    return(userlist)

def multiname(*usernames):
    for name in usernames:
        listmaker(name)
        listoffilmnames.append(filmlist)
        print(name)

def process_club_list(user_list,others):
    merge_frames = others.merge(user_list,on=['title','most_checked'],how='left')
    not_same_titles = merge_frames[merge_frames.top_lists_y.isnull()]
    del not_same_titles['top_lists_y']
    not_same_titles.columns=['most_checked', 'title', 'top_lists']
    return(not_same_titles)

def isinlist(fromlist,tolist):
    fromlist=fromlist['title'].tolist()
    tolist=tolist['title'].tolist()
    if set(fromlist).issubset(set(tolist)):
        return True
    else:
        return False
    
def iseven(number):
    if number %2==0:
        return True
    else:
        return False

def lookupinlist(newtitle):
    if newframe.loc[newframe['title'] == newtitle]:
        return True
    else:
        return False

def final_table(username,length):
    by_checked=club1_list.sort(['most_checked'],ascending=False)
    by_toplists=club1_list.sort(['top_lists'],ascending=False)
    finaldata=DataFrame(columns=('most_checked','title','top_lists'))
    for counter in range(0, length):
        if iseven(counter) is True:
            rec = by_checked.iloc[[counter/2]]
            if counter == 0:
                finaldata=finaldata.append(rec,ignore_index=True)
            else:
                if isinlist(rec,finaldata) is True:
                    continue
                else:
                    finaldata=finaldata.append(rec,ignore_index=True)
        else:
            rec = by_toplists.iloc[[(counter-1)/2]]
            if isinlist(rec,finaldata) is True:
                continue
            else:
                finaldata=finaldata.append(rec,ignore_index=True)
    return finaldata


#this is working on a function that will do everything
def Listmaker_pro(usr1,pssword,*usrs):
    username1=usr1
    username2=usrs[0]
    username3=usrs[1]
    password=pssword
    csvgetter(username1,password,username2,username3)
    user1_list=listmaker(username1)
    user2_list=listmaker(username2)
    user3_list=listmaker(username3)
    combo_1_2=user1_list.append(user2_list)
    seenby_1_2=combo_1_2[combo_1_2.duplicated('title')==True]
    combo_1_3=user1_list.append(user3_list)
    seenby_1_3=combo_1_3[combo_1_3.duplicated('title')==True]
    combo_2_3=user2_list.append(user3_list)
    seenby_2_3=combo_2_3[combo_2_3.duplicated('title')==True] 
    club1_list=process_club_list(user1_list,seenby_2_3) 
    club2_list=process_club_list(user2_list,seenby_1_3)
    club3_list=process_club_list(user3_list,seenby_1_2)
    final_table(usr1,20)

In [2]:
username1='hermesmovies'
username2='Haessar'
username3='James88'
password=''

In [3]:
csvgetter(username1,password,username2,username3)

There are 3 users in the database: 
Haessar_checklist.csv
James88_checklist.csv
hermesmovies_checklist.csv


In [4]:
user1_list=listmaker(username1)
user2_list=listmaker(username2)
user3_list=listmaker(username3)
print("user1 is "+username1+", user2 is "+username2+" and user3 is "+username3+", user#_list is a dataframe of each users list")

user1 is hermesmovies, user2 is Haessar and user3 is James88, user#_list is a dataframe of each users list


In [9]:
print(username1)

hermesmovies


Want to make a list that both james and will have seen that I haven't seen

In [10]:
#join User 1 and User 2
combo_1_2=user1_list.append(user2_list)
#return duplicates
seenby_1_2=combo_1_2[combo_1_2.duplicated('title')==True]

#join User 1 and User 3
combo_1_3=user1_list.append(user3_list)
seenby_1_3=combo_1_3[combo_1_3.duplicated('title')==True]

#join User 2 and User 3
combo_2_3=user2_list.append(user3_list)
seenby_2_3=combo_2_3[combo_2_3.duplicated('title')==True] 

In [11]:
#Club list for User 1:
club1_list = process_club_list(user1_list,seenby_2_3) 

#Club list for User 2:
club2_list = process_club_list(user2_list,seenby_1_3)

#Club list for User 3:
club3_list = process_club_list(user3_list,seenby_1_2)

In [12]:
final_table("Haessar",20)

,most_checked,title,top_lists
0,42273,Twilight,1
1,10365,The Wicker Man,12
2,40277,Sleepy Hollow,3
3,15944,12 Years a Slave,12
4,36817,Twister,2
5,35561,The Others,6
6,26309,Låt den rätte komma in,12
7,34486,Face/Off,4
8,6585,Withnail & I,11
9,33601,Terminator 3: Rise of the Machines,1


In [6]:
Listmaker_pro('hermesmovies','1xdavex1','James88','Haessar')

There are 3 users in the database: 
James88_checklist.csv
Haessar_checklist.csv
hermesmovies_checklist.csv


NameError: global name 'club1_list' is not defined